<a href="https://colab.research.google.com/github/Yewon9/STT_JEJU/blob/main/EDA_and_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install pydub

In [9]:
import pandas as pd
from pydub import AudioSegment
from tqdm import tqdm
import matplotlib.pyplot as plt
import librosa
import numpy as np
import seaborn as sns
import re

# DATA

In [3]:
df = pd.read_csv('df.csv')

In [4]:
df

,Path,Text,Standard Text
0,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,여름에 보양식은 그 동네에서 보면 그 자리들 잡아가지고 된장에 풀어가지고 생된장에 ...,여름에 보양식은 그 동네에서 보면 그 자리돔들 잡아서 된장에 풀어서 생된장에 풀어서...
1,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,태풍은 나 중학교 땐가 정확하게 기억은 잘 안 나는디 옛날에 해가지고는 엄청나게 사...,태풍은 나 중학교 때인가 정확하게 기억은 잘 안 나는데 옛날에 해가지고는 엄청나게 ...
2,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,그추룩헹은에 이제 제사 지낸 지가 한 이제 오 년 정도 됐신디예 어떵 하다 보니까 ...,그렇게 해서 이제 제사 지낸 지가 한 이제 오 년 정도 됐는데요 어떻게 하다 보니까...
3,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,어 여기는 지금 농작물은 농사를 뭐 밭농사랑 뭐 과수원 같은 거 많이 하니까 뭐 이...,어 여기는 지금 농작물은 농사를 뭐 밭농사랑 뭐 과수원 같은 거 많이 하니까 뭐 이...
4,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,우리는 형제 자매가 많아 가지고 옛날이라부난 옛날에 어른들은 게 아이들 형제들 네 ...,우리는 형제 자매가 많아 가지고 옛날이다보니까 옛날에 어른들은 게 아이들 형제들 네...
...,...,...,...
12337,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,이제 눈 어두웡 바농귀 끼지 못행 바농질 못 하켜,이제 눈 어두워서 바늘귀 꿰지 못해서 바느질 못 하겠어
12338,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,밧디서 나록을 비민 무껑 그 나록뭇을 쉐에 시껑 집이 왓저,밭에서 벼를 베면 묶어서 그 볏단을 소에 싣고 집에 왔지
12339,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,일 끗나거덜랑 어드레 가지 말앙 고짝 집더레 옵서예,일 끝나거든 어디 가지 말고 곧장 집으로 오세요
12340,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,나가 그자 온 것이 아니고 이 집이 큰똘을 중진허젠 허로 왓수다,내가 그냥 온 것이 아니고 이 집이 큰딸을 중신하려고 하러 왔습니다


## 데이터 개수 확인

In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12342 entries, 0 to 12341
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Path           12342 non-null  object
 1   Text           12342 non-null  object
 2   Standard Text  12342 non-null  object
dtypes: object(3)
memory usage: 289.4+ KB
None


In [6]:
print(df.describe())

                                                     Path  \
count                                               12342   
unique                                              12342   
top     /content/drive/MyDrive/JEJU/data/wav/TL_02/say...   
freq                                                    1   

                           Text        Standard Text  
count                     12342                12342  
unique                     8771                 8413  
top     느네 아방이영 어멍이영 혼디 어디 감신고이  우리 아들은 거짓말을 모르는 아이야  
freq                         36                   37  


In [7]:
missing_text = df[df['Text'] == '']
print(f"빈 텍스트 개수: {len(missing_text)}")

빈 텍스트 개수: 0


## 음성 파일 길이 분석

In [10]:
def get_audio_length(path):
  audio = AudioSegment.from_wav(path)
  return len(audio) / 1000

In [ ]:
df['Audio_Length'] = [get_audio_length(path) for path in tqdm(df['Path'])]

  2%|▏         | 303/12342 [02:07<1:21:19,  2.47it/s]

In [ ]:
plt.figure(figsize = (10, 6))
plt.hist(df['Audio_Length'], bins = 30, color = 'blue', alpha = 0.7)
plt.title('Distribution of Audio Lengths')
plt.xlabel('Audio Length (seconds)')
plt.ylabel('Frequency')
plt.show()

## 텍스트 길이 분석

In [ ]:
df['Text_Length'] = df['Text'].apply(len)

In [ ]:
print(df[['Text', 'Text_Length']].head())

In [ ]:
plt.figure(figsize = (10, 6))
plt.hist(df['Text_Length'], bins = 30, color = 'blue', alpha = 0.7)
plt.title('Distribution of Text Lengths')
plt.xlabel('Text Length (characters)')
plt.ylabel('Frequency')
plt.show()

## 빈 텍스트 확인

In [ ]:
empty_texts = df[df['Text_Length'] == 0]
print(f"빈 텍스트 개수: {len(empty_texts)}")

In [ ]:
df.to_csv('df_eda.csv', index = False, encoding = 'utf-8-sig')